In [97]:
from datetime import datetime
import json
import os
import requests

In [101]:
with open(os.path.expanduser('~/Dropbox/Solid Internet Properties/shopify_api_creds.json'), 'r') as f:
    api_creds = json.load(f)
HOST = "https://pinatas.myshopify.com"
orders = "/admin/orders.json?"
unshipped = "fulfillment_status=unshipped"

response = requests.get(HOST + orders,
                        auth=(api_creds["API_KEY"],
                              api_creds["API_PASSWORD"]))

In [102]:
orders = json.loads(response.text)["orders"]

In [103]:
def parse_order(order):

    order_obj = {
        "date": datetime.today().strftime("%m/%d/%Y"),
        "qty": None,
        "pinata": None,
        "busters": None,
        "blindfolds": None,
        "pullstrings": None,
        "rushed": None,
        "pictures": None,
        "size": None,
        "pinata_notes": None,
        "additional_notes": None, # TODO: what's the difference?
        "price": None,
        "party_date": None
    }

    order_obj["additional_notes"] = order["note"]
    for note_attr in order["note_attributes"]:
        if note_attr["name"] == "date":
            order_obj["party_date"] = note_attr["value"]

    for item in order['line_items']:
        if item['title'] == "The Best Custom Pinata":
            order_obj["qty"] = item['quantity']
            order_obj["size"] = item["variant_title"]
            for prop in item["properties"]:
                if prop["name"] == "image":
                    order_obj["pictures"] = prop["value"]
                elif prop["name"] == "description":
                    order_obj["pinata_notes"] = prop["value"]

        elif item["title"] == "Buster Bat Only":
            order_obj["busters"] = item['quantity']

        elif item["title"] == "Pinata Buster and Blindfold Combo":
            order_obj["busters"] = item['quantity']
            order_obj["blindfolds"] = item['quantity']
            
        else:
            print "\nMISSED ITEM: " + str(item["name"])
            print json.dumps(item, indent=2)

    return order_obj

In [107]:
order = orders[0]
#parse_order(order)